In [104]:
%pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [26]:
import os
import streamlit as st
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace,HuggingFaceEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings,ChatGoogleGenerativeAI
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS,Chroma
from langchain_groq import ChatGroq
from dotenv import load_dotenv


In [27]:
# loading .env
load_dotenv()

True

In [28]:
# model 

# llm=HuggingFaceEndpoint(
#     model='deepseek-ai/DeepSeek-R1',
#     task='text-generation',
#     temperature=0
# )

# model=ChatHuggingFace(llm=llm)

# model = ChatGroq(
#         model="llama3-8b-8192",
#         temperature=0
#     )

model=ChatGoogleGenerativeAI(model='gemini-2.5-pro')


In [29]:
# parser
parser=StrOutputParser()

In [30]:
# loading
loader=PyMuPDFLoader(file_path=r'C:\Users\Issac Chowdary\Downloads\projects\pdfchatbot\MixtureOfRecursions-Bygoogle.pdf')
docs=loader.load()

print(docs[0])

page_content='Mixture-of-Recursions: Learning Dynamic Recursive
Depths for Adaptive Token-Level Computation
Sangmin Bae1,*, Yujin Kim1,*, Reza Bayat2,*,
Sungnyun Kim1, Jiyoun Ha3, Tal Schuster4, Adam Fisch4, Hrayr Harutyunyan5, Ziwei Ji4,
Aaron Courville2,6,† and Se-Young Yun1,†
1KAIST AI, 2Mila, 3Google Cloud, 4Google DeepMind, 5Google Research, 6Université de Montréal
*Equal Contribution, †Corresponding Authors
Abstract: Scaling language models unlocks impressive capabilities, but the accompanying computational
and memory demands make both training and deployment expensive. Existing efficiency efforts typically
target either parameter sharing or adaptive computation, leaving open the question of how to attain both
simultaneously. We introduce Mixture-of-Recursions (MoR), a unified framework that combines the two axes
of efficiency inside a single Recursive Transformer. MoR reuses a shared stack of layers across recursion steps
to achieve parameter efficiency, while lightweight router

In [31]:
# text splitting

splitter=RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=0,
    
)

splitted_docs=splitter.split_documents(docs)

print(splitted_docs[0])
print(len(splitted_docs))

page_content='Mixture-of-Recursions: Learning Dynamic Recursive
Depths for Adaptive Token-Level Computation
Sangmin Bae1,*, Yujin Kim1,*, Reza Bayat2,*,
Sungnyun Kim1, Jiyoun Ha3, Tal Schuster4, Adam Fisch4, Hrayr Harutyunyan5, Ziwei Ji4,
Aaron Courville2,6,† and Se-Young Yun1,†
1KAIST AI, 2Mila, 3Google Cloud, 4Google DeepMind, 5Google Research, 6Université de Montréal
*Equal Contribution, †Corresponding Authors
Abstract: Scaling language models unlocks impressive capabilities, but the accompanying computational
and memory demands make both training and deployment expensive. Existing efficiency efforts typically
target either parameter sharing or adaptive computation, leaving open the question of how to attain both
simultaneously. We introduce Mixture-of-Recursions (MoR), a unified framework that combines the two axes
of efficiency inside a single Recursive Transformer. MoR reuses a shared stack of layers across recursion steps
to achieve parameter efficiency, while lightweight router

In [32]:
# embedding model
embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')




In [33]:
# vector_store
store=FAISS.from_documents(
    embedding=embeddings,
    documents=splitted_docs
)

In [34]:
# retriever
base_retriever=store.as_retriever(search_type='mmr',search_kwargs={'k':2})

compressor=LLMChainExtractor.from_llm(llm=model)

retriever=ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)


In [35]:
# template

template=PromptTemplate(
    template='You are a helpful large language model expert and helpful assistant, Answer from the provided transcript context.use your intelligence to explain it based on {question} and {context} in a helful and informative way.Give them related topics and page number of {context}. If the {context} is insufficient, just say you do not know \n{context} Question:{question}',
    input_variables=['context','question']
)

In [36]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda

parallel_chain=RunnableParallel({
    'context':retriever,
    'question':RunnablePassthrough()
})

chain=parallel_chain | template | model | parser



In [37]:
result=chain.invoke('what is ms dhoni')

print(result)

Based on the provided transcript context `[]`, **I do not know**.

The context you provided is empty and does not contain any information about MS Dhoni.

### Explanation

To answer your question about who MS Dhoni is, I need a transcript or document that contains information about him. Since the provided context `[]` is empty, I cannot generate an explanation.

### Related Topics and Page Numbers

Because the provided context is empty, I cannot identify any related topics or provide page numbers.

If you provide the transcript context that mentions MS Dhoni, I would be happy to analyze it for you and answer your question based on that information.
